In [6]:
# Import necessary libraries
import pandas as pd
import glob
from pandas_profiling import ProfileReport
import numpy as np
import json
import duckdb

In [7]:
# Read files from movielens, source: https://grouplens.org/datasets/movielens/
ratings = pd.read_csv("data/ml-25m/ratings.csv", sep = ",", index_col='movieId')
links = pd.read_csv("data/ml-25m/links.csv", sep = ",", index_col='movieId')
genres = pd.read_csv("data/ml-25m/movies.csv", sep=",", index_col='movieId')

In [14]:
# Split genres in sub genres and replace no genre listed with a NaN value
genres = pd.concat([genres, genres['genres'].str.split('|', expand=True)], axis=1)
genres = genres.replace('(no genres listed)', np.nan)    
genres.head()

,title,genres,0,1,2,3,4,5,6,7,...,0,1,2,3,4,5,6,7,8,9
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,...,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
2,Jumanji (1995),Adventure|Children|Fantasy,Adventure,Children,Fantasy,None,None,None,None,None,...,Adventure,Children,Fantasy,None,None,None,None,None,None,None
3,Grumpier Old Men (1995),Comedy|Romance,Comedy,Romance,None,None,None,None,None,None,...,Comedy,Romance,None,None,None,None,None,None,None,None
4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy,Drama,Romance,None,None,None,None,None,...,Comedy,Drama,Romance,None,None,None,None,None,None,None
5,Father of the Bride Part II (1995),Comedy,Comedy,None,None,None,None,None,None,None,...,Comedy,None,None,None,None,None,None,None,None,None


In [15]:
# NaN genre
genres.isnull().sum()

title         0
genres     5062
0          5062
1         30631
2         48957
3         58809
4         61593
5         62273
6         62396
7         62420
8         62422
9         62422
0          5062
1         30631
2         48957
3         58809
4         61593
5         62273
6         62396
7         62420
8         62422
9         62422
dtype: int64

# merging datasets

In [16]:
# Merging genre and rating dataset with link dataset. Only include 4 first genres
# Reset index for data
movielens_df = links
movielens_df['rating'] = ratings['rating'].groupby('movieId').mean()
movielens_df['year'] = genres['title'].str.extract(r'([0-9]{4})')
movielens_df['title'] = genres['title'].str[:-7]
movielens_df['genre_1'] = genres[0]
movielens_df['genre_2'] = genres[1]
movielens_df['genre_3'] = genres[2]
movielens_df['genre_4'] = genres[3]
movielens_df = movielens_df.reset_index()
movielens_df

ValueError: Columns must be same length as key

In [ ]:
# Perform EDA report on movielens data
profile = ProfileReport(movielens_df, title="Report")
profile

In [19]:
# Create a connection to a file called 'file.db'
con = duckdb.connect('imdb_reviews.db')

# Drop table if exits
#con.sql("DROP TABLE IF EXISTS movielens_information")


# Create a table and load data into it
con.sql("CREATE TABLE IF NOT EXISTS movielens_information AS SELECT * FROM movielens_df")

# Stop de connection
con.close()

In [18]:
con.close()


NameError: name 'con' is not defined